In [ ]:
import sys as _sys
import os

current_path = os.path.abspath(os.getcwd())

split = current_path.split("sgi_igl")
if len(split)<2:
    print("Please rename the repository 'sgi_igl'")
    raise ValueError
path_to_python_scripts = os.path.join(split[0], "sgi_igl/python/")
path_to_notifications = os.path.join(split[0], "sgi_igl/notebooks/notifications/")
path_to_settings = os.path.join(split[0], "sgi_igl/python/experiments/")
path_to_cubic_splines = os.path.join(split[0], "sgi_igl/ext/torchcubicspline/")
path_to_output = os.path.join(split[0], "sgi_igl/output/")
path_to_output_snake = os.path.join(path_to_output, "snake_forward/")

if not os.path.exists(path_to_output_snake):
    os.makedirs(path_to_output_snake)

_sys.path.insert(0, path_to_python_scripts)
_sys.path.insert(0, path_to_settings)
_sys.path.insert(0, path_to_cubic_splines)

In [ ]:
import numpy as np
import shutil
import torch

from geometry_io import export_snakes_to_json
from snake_shapes import snake_angles_generation
from step_forward_edge import multiple_steps_forward
from vis_utils_snake import plot_animated_snake
from vis_utils import produce_video_from_path

TORCH_DTYPE = torch.float64
torch.set_default_dtype(TORCH_DTYPE)

# Define initial gait and material parameters

## Parameters

In [ ]:
n_points_snake, n_ts = 11, 80
n_angles = n_points_snake - 2
edges = np.array([[idv, idv+1] for idv in range(n_points_snake-1)])
n_edges = edges.shape[0]
rho, eps, snake_length, close_snake_gait = 1.0e-2, 4.0e-2, 1.0, True
# All the joints are active
broken_joint_ids, broken_joint_angles = [], torch.tensor([])
init_perturb_magnitude = 1.0e-1

broken_joint_angles = broken_joint_angles.reshape(1, -1) * torch.ones(size=(n_ts-close_snake_gait, 1))
n_op_angles = n_angles - len(broken_joint_ids)

edges_torch = torch.tensor(edges)
# Define the function to compute the anisotropy direction: The tangent vectors for snakes
def fun_anisotropy_dir(x):
    tangents = x[..., edges_torch[:, 1], :] - x[..., edges_torch[:, 0], :]
    tangents = tangents / torch.linalg.norm(tangents, dim=-1, keepdims=True)
    return tangents

## Initial gait

We set the angles to random values, which results in a jagged shape sequence. You can replace it with any sequence you like.

In [ ]:
# Define the operational angles sequence
operational_angles = torch.zeros(n_ts-close_snake_gait, n_op_angles)
torch.manual_seed(0)
operational_angles += init_perturb_magnitude * torch.randn(n_ts-close_snake_gait, n_op_angles)

example_pos_ = torch.zeros(size=(n_ts, n_points_snake, 3))
pos_ = snake_angles_generation(
    operational_angles, snake_length, broken_joint_ids, broken_joint_angles,
    example_pos_, n_ts, close_gait=close_snake_gait
).numpy()

tangents_ = fun_anisotropy_dir(torch.tensor(pos_)).numpy()

edge_lengths = np.linalg.norm(pos_[:, 1:] - pos_[:, :-1], axis=-1)
masses = rho * edge_lengths
a_weights = np.ones(shape=(n_ts, n_edges))
b_weights = (eps - 1.0) * np.ones(shape=(n_ts, n_edges))

force_0 = np.zeros(shape=(3,))
torque_0 = np.zeros(shape=(3,))

In [ ]:
path_to_images_anim = os.path.join(path_to_output_snake, "images_anim/")

# clear existing images
if os.path.exists(path_to_images_anim):
    shutil.rmtree(path_to_images_anim)
os.makedirs(path_to_images_anim)

plot_animated_snake(
    pos_, path_to_images_anim,
    g=None, gt=None, gcp=None,
    exponent=1.0, xy_lim=None, 
    show_orientation=False, show_snake_trail=False, 
    show_g_trail=True, show_g_start=True,
    arrow_params=None, min_aspect_ratio=1.0,
    padding_mult_xy=np.array([0.05, 0.2]),
)

In [ ]:
fn_pattern = os.path.join(path_to_images_anim, "step_%05d.png")
produce_video_from_path(
    fn_pattern, path_to_output_snake, 
    "forward_pos_.mp4", overwrite_anim=True, transparent=False
)

# Compute trajectory and save

Run the forward simulation using your shape sequence and save the resulting trajectory.

In [ ]:
pos, tangents, g = multiple_steps_forward(
    pos_, tangents_, masses, a_weights, b_weights, edges, force_0, torque_0, g0=np.array([0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])
)

save_path = os.path.join(path_to_output_snake, "forward.json")

export_snakes_to_json(
    pos_, g, pos, force_0, torque_0, save_path, edges=None,
    weights_optim=None, quantities_per_vertex=None,
    quantities_per_edge=None, target_final_g=None,
)

In [ ]:
path_to_images_anim = os.path.join(path_to_output_snake, "images_traj_anim/")

# clear existing images
if os.path.exists(path_to_images_anim):
    shutil.rmtree(path_to_images_anim)
os.makedirs(path_to_images_anim)

plot_animated_snake(
    pos, path_to_images_anim,
    g=g, gt=None, gcp=None,
    exponent=1.0, xy_lim=None, 
    show_orientation=False, show_snake_trail=False, 
    show_g_trail=True, show_g_start=True,
    arrow_params=None, min_aspect_ratio=1.0,
    padding_mult_xy=np.array([0.05, 0.2]),
)

In [ ]:
fn_pattern = os.path.join(path_to_images_anim, "step_%05d.png")
produce_video_from_path(
    fn_pattern, path_to_output_snake, 
    "forward_pos.mp4", overwrite_anim=True, transparent=False
)